### Purpose of this notebook

This notebook is designed to demonstrate how to process TSSE spreadsheets once you have created them (as done in the `01_....ipynb` file and filled them in (using Excel)

### Let's go!

In [22]:
from pathlib import Path
from tsse_data.toc_measurement import process_toc_spreadsheet

In [23]:
process_toc_spreadsheet?

Signature:
process_toc_spreadsheet(
    filepath: str,
    dims: list,
    amine: dict,
    common_dims: dict = None,
    calib=None,
)
Docstring:
filepath: str
    The filepath to the spreadsheet to be read and evaluated.

dims: list
    The list of column names from the spreadsheet that you wish to use as dimension labels.

amine: dict
    This dict must contain two key-value pairs:
        nc, representing the number of carbon atoms in the molecule
        mw, the molecular weight of the molecule.

common_dims: dict
    Common dims is used to add dimensions common to all measurements in this dataset.
    Each key is a dimension name, and each val is the value to be written to that dimension.

    For example, if all of these measurements were done using diisopropylamine and at 25 C, an argument
    common_dims = {'amine': 'dipa', 'T': 25}
    could be included.

calib: function, default None
    The calibration function that you wish to apply to the TOC. The TOC is regularly calibra

Create and examine the path to the folder containing the spreadsheets. Ordinarily the path you define here would be the same one used in the create_sheets notebook. However, to avoid the files being accidentally overwritten during the tutorial, a separate directory with filled spreadsheets has been provided.

In [51]:
p = Path.cwd() / 'measurements_filled'
p

PosixPath('/Users/ianbillinge/dev/tsse_data/tsse_data/tutorial/measurements_filled')

Define the dimensions that are described in the spreadsheet... as well as the common_dims, which are common descriptors that apply to the whole spreadsheet and therefore were not entered into the spreadsheet.

In [25]:
dims = ['amine', 'cation', 'anion', 'experimenter', 'replicate']
cd = {'T':25}
dipa = {'mw':101.19, 'nc':6}
calibration = None

Define the filename of the spreadsheet you will read.

In [26]:
filename = 'toc.xlsx'
path = str(p / filename)
path

'/Users/ianbillinge/dev/tsse_data/tsse_data/tutorial/measurements_filled/toc.xlsx'

In [35]:
ds = process_toc_spreadsheet(path, dims, dipa, common_dims=cd, calib=calibration)
ds

/Users/ianbillinge/dev/tsse_data/tsse_data/toc_measurement.py:111: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-5.389500000000001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_w_toc_calib[i] = b + m * log_w_toc[i]
/Users/ianbillinge/dev/tsse_data/tsse_data/toc_measurement.py:111: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-5.389500000000001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_w_toc_calib[i] = b + m * log_w_toc[i]


<xarray.Dataset>
Dimensions:            (amine: 1, cation: 1, anion: 1, experimenter: 1,
                        replicate: 1, T: 1)
Coordinates:
  * amine              (amine) object 'dipa'
  * cation             (cation) object 'Na'
  * anion              (anion) object 'Cl'
  * experimenter       (experimenter) object 'ib'
  * replicate          (replicate) int64 1
  * T                  (T) int64 25
Data variables:
    m_sample           (amine, cation, anion, experimenter, replicate, T) float64 ...
    m_water            (amine, cation, anion, experimenter, replicate, T) int64 ...
    toc_raw            (amine, cation, anion, experimenter, replicate, T) int64 ...
    dtoc_raw           (amine, cation, anion, experimenter, replicate, T) int64 ...
    w_toc_raw          (amine, cation, anion, experimenter, replicate, T) float64 ...
    dw_toc_raw         (amine, cation, anion, experimenter, replicate, T) float64 ...
    w_toc_calibrated   (amine, cation, anion, experimenter, replicate, T) float64 ...
    dw_toc_calibrated  (amine, cation, anion, experimenter, replicate, T) float64 ...
    w_toc_adj_dil      (amine, cation, anion, experimenter, replicate, T) float64 ...
    w_a                (amine, cation, anion, experimenter, T) float64 0.002296
    dw_a               (amine, cation, anion, experimenter, T) float64 0.0

The data can be saved in a variety of formats. Here, we create a new folder called `measurements_processed`.

In [48]:
out_dir = Path.cwd() / 'measurements_processed'

try:
    out_dir.mkdir()
except FileExistsError:
    pass

The file can be exported in a variety of formats. If you want to export as Excel, you can convert it to a Pandas DataFrame, then use Pandas's tools to export to excel:

In [52]:
df = ds.to_dataframe()
df.to_excel(out_dir / 'toc.xlsx')
df.to_csv(out_dir / 'toc.csv')
df

,,,,,,m_sample,m_water,toc_raw,dtoc_raw,w_toc_raw,dw_toc_raw,w_toc_calibrated,dw_toc_calibrated,w_toc_adj_dil,w_a,dw_a
amine,cation,anion,experimenter,replicate,T,,,,,,,,,,,
dipa,Na,Cl,ib,1,25,0.1,40,10000,100,0.00001,1.000000e-07,0.000004,0.000004,0.001635,0.002296,0.0


And if you want to export the data as a dataset (netcdf format, .nc), that can be done straightforwardly:


In [50]:
ds.to_netcdf(out_dir / 'toc.nc')